In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

In [2]:
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
import pandas as pd
import json
import torch
from tqdm import tqdm
from datasets import load_dataset
from peft import LoraConfig, PeftConfig
from datasets import Dataset
import copy

[2024-02-27 13:46:15,215] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [31]:
device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU
device

0

In [3]:
train_filename = "dataset/ace/train.txt"
val_filename = "dataset/ace/dev.txt"
test_filename = "dataset/ace/test.txt"

In [4]:
def load_data(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Split content into samples based on two newlines
    samples = content.strip().split('\n\n')

    # Create a list to store token-label pairs and sample IDs
    token_label_pairs = []
    sample_ids = []

    # Iterate through samples
    for sample in samples:
        lines = sample.split('\n')
        for line in lines:
            # Split each line into token and label
            token, label = line.split(' ')
            token_label_pairs.append((token, label))

        # Add the sample ID for each token in the current sample
        sample_ids.extend([len(sample_ids)] * len(lines))

    # Create a DataFrame
    df = pd.DataFrame(token_label_pairs, columns=['Token', 'Label'])

    # Add a column for sample ID
    df['Sample'] = sample_ids

    # Reset the index
    df.reset_index(drop=True, inplace=True)

    return df

def get_unique_labels(dataframe):
    # Create a set to store unique labels
    unique_labels = set()

    # Iterate through the "Label" column in the DataFrame
    for label in dataframe['Label']:
        # Extract the base label (ignoring the prefix B- or I-)
        base_label = label[2:] if label.startswith('B-') or label.startswith('I-') else label

        # Add the base label to the set
        unique_labels.add(base_label)

    return list(unique_labels)



In [5]:
train_df = load_data(train_filename)
val_df = load_data(val_filename)
test_df = load_data(test_filename)

In [6]:
unique_labels = get_unique_labels(train_df)
unique_labels.remove("O")
entity_dict = {label: [] for label in unique_labels}

In [7]:
def get_instruction(input_sentence,output):
    instruction = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Your task is a Named Entity Recognition (NER) task. Given a text , predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON payload. If no entity present then output will be empty json schema. 
Here are the entity labels with empty list. 
{json.dumps(entity_dict)}

Below is an example of text and the corresponding entities in json format.

### Input: 
{input_sentence}\n
### Output: 
{output}\n
### End"""
    return instruction

In [8]:
test_ins = "Below is the text enclosed in square brackets and find corresponding entities from the above labels in json output."

INSTRUCTION = f"""
Your task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON payload. If no entity present then output will be empty json schema. 
Here are the entity labels with empty list. 
{json.dumps(entity_dict)}

Below is the text enclosed in square brackets and the corresponding entities in json format.
"""

TEST_INSTRUCTION = f"""
Your task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON format. If no entity present then output will be empty json schema. 
Here are the entity labels with empty list. 
{json.dumps(entity_dict)}

Below is the text enclosed in square brackets and the corresponding entities in json format.
"""

In [9]:
entity_dict

{'Subject.Race': [],
 'Subject.Age': [],
 'Potential_therapeutic_event.Trigger': [],
 'Time_elapsed': [],
 'Duration': [],
 'Treatment.Freq': [],
 'Route': [],
 'Gender': [],
 'Drug': [],
 'Subject.Sub_Disorder': [],
 'Effect': [],
 'Treatment.Drug': [],
 'Adverse_event.Trigger': [],
 'Treat_Disorder': [],
 'Treatment.Route': [],
 'Dosage': [],
 'Freq': [],
 'Treatment.Combination.Drug': [],
 'Sub_Disorder': [],
 'Subject': [],
 'Race': [],
 'Subject.Gender': [],
 'Population': [],
 'Subject.Population': [],
 'Treatment.Time_elapsed': [],
 'Treatment.Treat_Disorder': [],
 'Combination.Drug': [],
 'Treatment.Dosage': [],
 'Treatment.Duration': [],
 'Treatment': []}

In [37]:
def create_instruction_samples(dataframe, entity_dict ,data_type="train"):
    instruction_samples = []
    output_dicts = []

    for sample_id, group in dataframe.groupby("Sample"):
        input_sentence = " ".join(group["Token"])
        
        output_dict = {label: [] for label in entity_dict}

        for _, row in group.iterrows():
            label = row["Label"]
            token = row["Token"]

            # Extract the base label (ignoring the prefix B- or I-)
            base_label = label[2:] if label.startswith('B-') or label.startswith('I-') else label

            # Categorize the token based on the base label
            if base_label in output_dict:
                output_dict[base_label].append(token)
        
        if data_type =="train":
            instruction = f"""{INSTRUCTION}\n[{input_sentence}] = {json.dumps(output_dict)}"""
        else :
            output = "{"
            instruction = f"""{TEST_INSTRUCTION}\n[{input_sentence}] = {output}"""
        # if data_type =="train":
        #     instruction = get_instruction(input_sentence,output_dict)
        # else:
        #     instruction = get_instruction(input_sentence,"{")
        instruction_samples.append(instruction)
        output_dicts.append(output_dict)

    # Create a DataFrame with a single column "text"
    df_instruction = pd.DataFrame({"text": instruction_samples})

    return df_instruction,output_dicts

In [38]:
train_instruction_df,train_outputs = create_instruction_samples(train_df,entity_dict)
val_instruction_df,val_outputs = create_instruction_samples(val_df,entity_dict)
test_instruction_df,test_outputs = create_instruction_samples(test_df,entity_dict,data_type="test")

In [12]:
train_instruction_df.shape

(3006, 1)

In [13]:
# train_instruction_df = train_instruction_df.sample(n=500)


In [14]:
train_instruction_df.shape

(3006, 1)

In [15]:
test_instruction_df.iloc[0]['text']

'\nYour task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON format. If no entity present then output will be empty json schema. \nHere are the entity labels with empty list. \n{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": [], "Treatment.Drug": [], "Adverse_event.Trigger": [], "Treat_Disorder": [], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": [], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": [], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": []}\n\nBelow is the text enclosed in square brackets and the 

In [16]:
train_instruction_df.iloc[0]['text']

'\nYour task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON payload. If no entity present then output will be empty json schema. \nHere are the entity labels with empty list. \n{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": [], "Treatment.Drug": [], "Adverse_event.Trigger": [], "Treat_Disorder": [], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": [], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": [], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": []}\n\nBelow is the text enclosed in square brackets and the

In [17]:
print(train_instruction_df.iloc[0]['text'])


Your task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON payload. If no entity present then output will be empty json schema. 
Here are the entity labels with empty list. 
{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": [], "Treatment.Drug": [], "Adverse_event.Trigger": [], "Treat_Disorder": [], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": [], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": [], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": []}

Below is the text enclosed in square brackets and the corre

In [18]:
print(test_instruction_df.iloc[0]['text'])


Your task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON format. If no entity present then output will be empty json schema. 
Here are the entity labels with empty list. 
{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": [], "Treatment.Drug": [], "Adverse_event.Trigger": [], "Treat_Disorder": [], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": [], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": [], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": []}

Below is the text enclosed in square brackets and the corres

In [19]:
test_instruction_df.head()["text"][0]

'\nYour task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON format. If no entity present then output will be empty json schema. \nHere are the entity labels with empty list. \n{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": [], "Treatment.Drug": [], "Adverse_event.Trigger": [], "Treat_Disorder": [], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": [], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": [], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": []}\n\nBelow is the text enclosed in square brackets and the 

In [20]:
train_data = Dataset.from_pandas(train_instruction_df)
eval_data = Dataset.from_pandas(val_instruction_df)
test_data = Dataset.from_pandas(test_instruction_df)

In [21]:
train_data['text'][0]

'\nYour task is a Named Entity Recognition (NER) task. Given a text, predict the Label of each entity, then place the entity into the list associated with the Label in an output JSON payload. If no entity present then output will be empty json schema. \nHere are the entity labels with empty list. \n{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": [], "Treatment.Drug": [], "Adverse_event.Trigger": [], "Treat_Disorder": [], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": [], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": [], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": []}\n\nBelow is the text enclosed in square brackets and the

In [22]:
# Load the tokenizer and model
model_name = "/home/jomondal/experiments/mywork/pretrained_models/Mixtral-8x7B-Instruct-v0.1"
trained_model_name = f"/home/jomondal/experiments/mywork/llama_exp/trained_models/json_{model_name.split('/')[-1]}"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [34]:
def extract_text_after_equals_with_braces(input_string):
    equal_index = input_string.find('=')
    # print(input_string[equal_index:])
    brace_index = input_string.find('{', equal_index)
    end_brace_index = input_string.find('}', brace_index)
    return input_string[brace_index:end_brace_index + 1]

def predict(data, model, tokenizer):
    y_pred = []
    pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 500, 
                        temperature = 0.001,
                       )
    texts = data['text']
    
    for i in tqdm(range(len(texts))):
        
        prompt = texts[i]
        # print(prompt)
        result = pipe(prompt,pad_token_id=tokenizer.eos_token_id)
        # print("Result:",result)
        answer = result[0]['generated_text']
        # print("Answer:",answer)
        answer = answer.replace('""','"')
        # print("Removed double quotes:",answer)
        answer = extract_text_after_equals_with_braces(answer)
        # answer = answer.replace('\n','')
        # print(answer)
        try:
            answer = json.loads(answer)
        except Exception as e:
            print("Result:",result)
            print("Answer:",answer)
            answer = entity_dict
        y_pred.append(answer)
    return y_pred

In [24]:
# def evaluate(gold,pred)

In [25]:
# y_pred = predict(test_data, model, tokenizer)

In [26]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4, # 4
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    # lr_scheduler_type="constant",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

Map:   0%|          | 0/3006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
# Train model
trainer.train()

/home/jomondal/experiments/mywork/venv/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.107100,0.117440
2,0.087000,0.113697
3,0.102600,0.112729


TrainOutput(global_step=564, training_loss=0.14197586741007812, metrics={'train_runtime': 8664.0757, 'train_samples_per_second': 1.041, 'train_steps_per_second': 0.065, 'total_flos': 1.5713731461422776e+18, 'train_loss': 0.14197586741007812, 'epoch': 3.0})

In [28]:
# Save trained model
trainer.model.save_pretrained(trained_model_name)
trainer.tokenizer.save_pretrained(trained_model_name)

/home/jomondal/experiments/mywork/venv/lib/python3.8/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in /home/jomondal/experiments/mywork/pretrained_models/Mixtral-8x7B-Instruct-v0.1 - will assume that the vocabulary was not modified.
  warnings.warn(


('/home/jomondal/experiments/mywork/llama_exp/trained_models/json_Mixtral-8x7B-Instruct-v0.1/tokenizer_config.json',
 '/home/jomondal/experiments/mywork/llama_exp/trained_models/json_Mixtral-8x7B-Instruct-v0.1/special_tokens_map.json',
 '/home/jomondal/experiments/mywork/llama_exp/trained_models/json_Mixtral-8x7B-Instruct-v0.1/tokenizer.model',
 '/home/jomondal/experiments/mywork/llama_exp/trained_models/json_Mixtral-8x7B-Instruct-v0.1/added_tokens.json',
 '/home/jomondal/experiments/mywork/llama_exp/trained_models/json_Mixtral-8x7B-Instruct-v0.1/tokenizer.json')

In [29]:
len(test_data['text'])

968

In [35]:
y_pred = predict(test_data, model, tokenizer)

 74%|████████████████████████████████████████████████████████████████████████████████████████████████████▎                                   | 714/968 [10:14:31<4:17:15, 60.77s/it]

{{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": ["adverse", "effects"], "Treatment.Drug": ["tolterodine"], "Adverse_event.Trigger": ["developed"], "Treat_Disorder": ["urinary", "disorders"], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": ["patients"], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": ["Two"], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": []}


 74%|█████████████████████████████████████████████████████████████████████████████████████████████████████▏                                  | 720/968 [10:19:12<3:06:01, 45.01s/it]

{"Subject.Race": [], "Subject.Age": [], "Potential_therapeutic_event.Trigger": [], "Time_elapsed": [], "Duration": [], "Treatment.Freq": [], "Route": [], "Gender": [], "Drug": [], "Subject.Sub_Disorder": [], "Effect": ["Severe", "adenovirus", "pneumonia", "(", "AVP", " )"], "Treatment.Drug": ["infliximab"], "Adverse_event.Trigger": ["following"], "Treat_Disorder": [], "Treatment.Route": [], "Dosage": [], "Freq": [], "Treatment.Combination.Drug": [], "Sub_Disorder": [], "Subject": [], "Race": [], "Subject.Gender": [], "Population": [], "Subject.Population": [], "Treatment.Time_elapsed": [], "Treatment.Treat_Disorder": [], "Combination.Drug": [], "Treatment.Dosage": [], "Treatment.Duration": [], "Treatment": ["infusion"", "for"", "the"", "treatment"", "of"", "Crohn", "'s", "disease"]}


 75%|█████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 723/968 [10:21:55<3:30:34, 51.57s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 966/968 [13:48:51<01:31, 45.99s/it]

{Subject.Race: [], Subject.Age: [], Potential_therapeutic_event.Trigger: [], Time_elapsed: [], Duration: [], Treatment.Freq: [], Route: [], Gender: [], Drug: [], Subject.Sub_Disorder: [], Effect: ['pulmonary', 'toxicity'], Treatment.Drug: ['CCNU'], Treatment.Route: [], Dosage: [], Freq: [], Treatment.Combination.Drug: [], Sub_Disorder: [], Treatment.Dosage: [], Treatment.Duration: [], Treatment: []}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 967/968 [13:49:55<00:51, 51.48s/it]

{Subject.Race: [], Subject.Age: [], Potential_therapeutic_event.Trigger: [], Time_elapsed: [], Duration: [], Treatment.Freq: [], Route: [], Gender: [], Drug: [], Subject.Sub_Disorder: [], Effect: ['hypoglycemia'], Treatment.Drug: ['clarithromycin'], Treatment.Route: [], Dosage: [], Freq: [], Treatment.Combination.Drug: [], Sub_Disorder: [], Treatment.Dosage: [], Treatment.Duration: [], Treatment: ['Concomitant', 'use', 'of', 'or', 'other', 'potent', 'inhibitors', 'of', 'CYP3A4', 'with','repaglinide']}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 968/968 [13:50:26<00:00, 51.47s/it]


In [ ]:
len(y_pred)

In [41]:
# Function to save dictionaries to JSONL file
def save_to_jsonl(file_path, list_of_dicts):
    with open(file_path, 'w') as f:
        for dictionary in list_of_dicts:
            f.write(json.dumps(dictionary) + '\n')

# File path to save JSONL file
pred_file_path = "outputs/test_output_predict.jsonl"

# Save list of dictionaries to JSONL file
save_to_jsonl(pred_file_path, y_pred)

In [42]:
gold_file_path = "outputs/test_output_gold.jsonl"
save_to_jsonl(gold_file_path, test_outputs)

### Evaluation

In [44]:
import json

def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    return data



In [48]:
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(gold, predict):
    # Initialize dictionaries to store true positives, false positives, and false negatives for each label
    tp = defaultdict(int)
    fp = defaultdict(int)
    fn = defaultdict(int)
    
    # Iterate over each document in gold and predicted data
    for gold_doc, predict_doc in zip(gold, predict):
        # Iterate over each NER label in the document
        for label in gold_doc.keys():
            # Calculate true positives
            tp[label] += len(set(predict_doc.get(label, [])) & set(gold_doc[label]))
            # Calculate false positives
            fp[label] += len(set(predict_doc.get(label, [])) - set(gold_doc[label]))
            # Calculate false negatives
            fn[label] += len(set(gold_doc[label]) - set(predict_doc.get(label, [])))
    
    # Initialize dictionaries to store precision, recall, and f1-score for each label
    precision = {}
    recall = {}
    f1 = {}
    
    # Calculate precision, recall, and f1-score for each label
    for label in tp.keys():
        precision[label] = tp[label] / (tp[label] + fp[label]) if (tp[label] + fp[label]) > 0 else 0
        recall[label] = tp[label] / (tp[label] + fn[label]) if (tp[label] + fn[label]) > 0 else 0
        f1[label] = 2 * precision[label] * recall[label] / (precision[label] + recall[label]) if (precision[label] + recall[label]) > 0 else 0
    
    # Calculate macro-averaged precision, recall, and f1-score for labels
    macro_precision = sum(precision.values()) / len(precision)
    macro_recall = sum(recall.values()) / len(recall)
    macro_f1 = sum(f1.values()) / len(f1)
    
    # Calculate overall micro-averaged precision, recall, and f1-score
    overall_tp = sum(tp.values())
    overall_fp = sum(fp.values())
    overall_fn = sum(fn.values())
    overall_precision = overall_tp / (overall_tp + overall_fp) if (overall_tp + overall_fp) > 0 else 0
    overall_recall = overall_tp / (overall_tp + overall_fn) if (overall_tp + overall_fn) > 0 else 0
    overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
    
    # Return results
    return {
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Macro Precision': macro_precision,
        'Macro Recall': macro_recall,
        'Macro F1': macro_f1,
        'Overall Micro Precision': overall_precision,
        'Overall Micro Recall': overall_recall,
        'Overall Micro F1': overall_f1
    }
gold_data = read_jsonl(gold_file_path)
predict_data = read_jsonl(pred_file_path)

evaluation_results = evaluate(gold_data, predict_data)

print("Label-specific Metrics:")
print("Label\tPrecision\tRecall\tF1")
for label in evaluation_results['Precision'].keys():
    precision = round(evaluation_results['Precision'][label], 2)
    recall = round(evaluation_results['Recall'][label], 2)
    f1 = round(evaluation_results['F1'][label], 2)
    print(f"{label}\t{precision}\t{recall}\t{f1}")

print()
micro_precision = round(evaluation_results['Overall Micro Precision'], 2)
micro_recall = round(evaluation_results['Overall Micro Recall'], 2)
micro_f1 = round(evaluation_results['Overall Micro F1'], 2)
print("Overall Micro Metrics:")
print(f"Micro Precision: {micro_precision}")
print(f"Micro Recall: {micro_recall}")
print(f"Micro F1-score: {micro_f1}")

print()
macro_precision = round(evaluation_results['Macro Precision'], 2)
macro_recall = round(evaluation_results['Macro Recall'], 2)
macro_f1 = round(evaluation_results['Macro F1'], 2)
print("Overall Macro Metrics:")
print(f"Macro Precision: {macro_precision}")
print(f"Macro Recall: {macro_recall}")
print(f"Macro F1-score: {macro_f1}")

Label-specific Metrics:
Label	Precision	Recall	F1
Subject.Race	1.0	0.36	0.53
Subject.Age	0.99	0.59	0.74
Potential_therapeutic_event.Trigger	0.44	0.11	0.18
Time_elapsed	0	0.0	0
Duration	0	0.0	0
Treatment.Freq	0	0.0	0
Route	0	0.0	0
Gender	0	0	0
Drug	0	0.0	0
Subject.Sub_Disorder	0.52	0.11	0.18
Effect	0.84	0.53	0.65
Treatment.Drug	0.92	0.59	0.72
Adverse_event.Trigger	0.69	0.46	0.55
Treat_Disorder	0.7	0.47	0.56
Treatment.Route	0.59	0.43	0.5
Dosage	0	0.0	0
Freq	0	0.0	0
Treatment.Combination.Drug	0.67	0.42	0.52
Sub_Disorder	0	0.0	0
Subject	0.66	0.43	0.52
Race	0	0	0
Subject.Gender	0.94	0.54	0.69
Population	0	0	0
Subject.Population	0.68	0.42	0.52
Treatment.Time_elapsed	0.41	0.22	0.29
Treatment.Treat_Disorder	0.0	0.0	0
Combination.Drug	0	0.0	0
Treatment.Dosage	0.75	0.4	0.52
Treatment.Duration	0.57	0.42	0.48
Treatment	0.56	0.37	0.45

Overall Micro Metrics:
Micro Precision: 0.75
Micro Recall: 0.46
Micro F1-score: 0.57

Overall Macro Metrics:
Macro Precision: 0.4
Macro Recall: 0.23
Macro F1-score: 